**Problem set 1**
**Subject: IO**
**Start Date 01/12/2025**

**Problem 1**

In [10]:
using CSV, DataFrames

#Input path
data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year2/term1/io/part2/data_ps1.csv" ##-->CHANGE JUST THIS PART MANUALLY!

#Output path
export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/" ##Change this manually too

main = CSV.read(data_path, DataFrame)

Row,yr,prefid,firmid,m,k,age,acquired,cost_m,q,rev,l,cost_l,cost_k
,Int64,Int64,Int64,Int64?,Int64?,Int64,Int64,Float64?,Float64,Float64,Int64?,Float64?,Float64?
1,1917,1,1,174866,120960,0,0,4.81021e5,1.55838e5,385355.0,1045968,49963.2,missing
2,1918,1,1,225000,120600,1,0,9.72158e5,412452.0,2.01e6,1359765,84420.0,missing
3,1919,1,1,225266,199248,2,0,1.16798e6,8.82063e5,5.14022e6,4956336,4.60354e5,missing
4,1920,1,1,383289,306440,3,0,2.14914e6,1.11663e6,5.15899e6,5278592,215085.0,missing
5,1896,1,2,33580,5772,18,0,38996.5,24626.3,55610.1,522144,2925.96,missing
6,1896,4,4,502159,112346,4,0,5.83157e5,3.97768e5,789222.0,8201556,53398.6,16698.8
7,1897,4,4,532756,144058,5,0,6.45381e5,461018.0,8.54804e5,8520094,78892.8,43161.2
8,1898,4,4,190027,49268,6,0,2.19785e5,150335.0,0.0,2707342,23877.5,30933.8
9,1896,16,5,843772,243090,7,0,9.79872e5,1.00331e6,1.35233e6,13692294,1.02388e5,39081.8


In [11]:
#1.a
using Statistics

#define variables

l= main.l
w= main.cost_l
rev= main.rev
k= main.k
r= main.cost_k
y= main.q

beta_l= ifelse.(rev.>0, (w) ./ (rev), missing) 

beta_l_total = mean(skipmissing(beta_l))
beta_l_std= std(skipmissing(beta_l))
beta_l_std= median(skipmissing(beta_l))



0.050841608736857224

In [12]:
#1.b 
using Statistics

total_cost = w .+  r

idx = findall(x -> !ismissing(x) && x > 0, total_cost)

beta_l_2 = (w[idx]) ./ total_cost[idx]


beta_l_total_2 = mean(beta_l_2)
beta_l_std_2= std(beta_l_2)
beta_l_std_2= median(beta_l_2)



0.5693124616887979

In [13]:
#1.c 

markup= rev[idx] ./ total_cost[idx]

markup_mean = mean(markup)
markup_std= std(markup)
markup_median= median(markup)

markup_mean


16.184801788871056

**Exercise 2**


In [36]:
# NEVER touch this again

using ShiftedArrays

main_raw = deepcopy(main)

function prepare_data(main_raw)
    df = deepcopy(main_raw)

    sort!(df, [:firmid, :yr])

    df = transform(
        groupby(df, :firmid),
        :l => (x -> ShiftedArrays.lag(x, 1)) => :l_lag1,
        :l => (x -> ShiftedArrays.lag(x, 2)) => :l_lag2,
        :k => (x -> ShiftedArrays.lag(x, 1)) => :k_lag1,
        :acquired => (x -> ShiftedArrays.lag(x, 1)) => :acquired_lag1,
        :acquired => (x -> ShiftedArrays.lag(x, 2)) => :acquired_lag2,
        :q => (x -> ShiftedArrays.lag(x, 1)) => :y_lag
    )

    needed = [:l, :k, :q, :l_lag1, :l_lag2, :k_lag1, :acquired_lag1, :acquired_lag2, :y_lag]
    df = dropmissing(df, needed)

    df = filter(row ->
        row.l > 0 &&
        row.k > 0 &&
        row.q > 0 &&
        row.y_lag > 0 &&
        row.l_lag1 > 0 &&
        row.l_lag2 > 0 &&
        row.k_lag1 > 0,
        df
    )

    return df
end

data = prepare_data(main_raw)


#make sure order is maintained of the vectors after sort
l= data.l
w= data.cost_l
rev= data.rev
k= data.k
r= data.cost_k
y= data.q

acquired_lag1= data.acquired_lag1
acquired_lag2=data.acquired_lag2
y_lag=data.y_lag


# turn variables into log
log_output= log.(y)
log_lagoutput= log.(y_lag)
log_l= log.(l)
log_k= log.(k)

log_l_lag1= log.(data.l_lag1)
log_l_lag2= log.(data.l_lag2)
log_k_lag1= log.(data.k_lag1)


# Matrix of Z

n = nrow(data)

Z = hcat(
    ones(n),
    log_l_lag1,
    log_k,
    log_k_lag1,
    log_l_lag2
)

n

901

In [23]:
using LinearAlgebra

#guess of values goes in theta. 
function gmm_estim(θ::AbstractVector, Z::AbstractMatrix, n)

    β0, βl, βk, ρ, βa= θ

    ω_t= @. log_output - β0 - βl*log_l - βk*log_k #nx1
    ω_t_lag= @. log_lagoutput- β0 - βl* log_l_lag1 -  βk*log_k_lag1 #nx1

    ν_hat= @. ω_t- ρ*ω_t_lag - βa*acquired_lag1 #nx1

    gn= (Z'*ν_hat)/n #kxn nx1-> kx1 moments

    W=I(length(gn))

    J = n * dot(gn, gn) #we minimize this to find θ
    return J

end 

gmm_estim (generic function with 1 method)

In [24]:
#Set reasonable starting values, starting from OLS:

X = hcat(
    ones(length(log_l)),
    log_l,
    log_k)
    


B_ols= (X'*X)\(X'*log_output)

3-element Vector{Float64}:
 -2.5425722270828772
  0.401442633297878
  0.822638353031784

In [37]:
using Optim

θ0 = [B_ols[1], B_ols[2], B_ols[3], 0.8, 0.5]   # example 

obj(θ) = gmm_estim(θ, Z, n)


res1_test = optimize(obj, θ0, NelderMead())
θ1_test = Optim.minimizer(res1_test)


5-element Vector{Float64}:
  2.943416173100644
  0.2714623910248015
  0.04365844446574626
  1.0089044981482558
 -1.7322986280322772

In [38]:
#two-step GMM, step 1.

function moments(θ::AbstractVector, Z::AbstractMatrix, n)

    β0, βl, βk, ρ, βa = θ

    ω_t = @. log_output - β0 - βl*log_l - βk*log_k
    ω_t_lag = @. log_lagoutput - β0 - βl*log_l_lag1 - βk*log_k_lag1

    ν_hat = @. ω_t - ρ*ω_t_lag - βa*acquired_lag1   # n×1

    g = (Z' * ν_hat) / n   # q×1 vector of moments

    return g, ν_hat
end


function gmm_objective(θ, Z, n, W)

    g, _=moments(θ, Z, n)
    return n*(g'*W*g)

end 

function s_robust(θ, Z, n)

    _, ν= moments(θ, Z, n)

    M= Z.*ν

    return (M'*M) /n

end 

q = size(Z, 2)
W1 = I(q)

obj1(θ) = gmm_objective(θ, Z, n, W1)
res1 = optimize(obj1, θ0, NelderMead())
θ1 = Optim.minimizer(res1)

S1 = s_robust(θ1, Z, n)
W2 = pinv(S1)

obj2(θ) = gmm_objective(θ, Z, n, W2)
res2 = optimize(obj2, θ0, NelderMead())
θ2 = Optim.minimizer(res2)



5-element Vector{Float64}:
   6.412615354447134
   0.2120851999009136
   0.10781040433528298
   1.0836148195846804
 -65.37003277619064

In [39]:
#standard errors


using ForwardDiff, LinearAlgebra

gfun(θ) = moments(θ, Z, n)[1]

D= ForwardDiff.jacobian(gfun,θ2)

A = D' * W2 * D
B = D' * W2 * S1 * W2 * D

S1 = 0.5 * (S1 + S1')
W2 = 0.5 * (W2 + W2')
A  = 0.5 * (A  + A')


V = inv(A) * B * inv(A) / n
se = sqrt.(diag(V))



5-element Vector{Float64}:
  0.8865807272150896
  0.05117596107657309
  0.055329090492150325
  0.025487295449717628
 21.080486530492223

In [21]:
Wsafe = I(size(Z,2))

A = D' * Wsafe * D
B = D' * Wsafe * S1 * Wsafe * D

V = inv(A) * B * inv(A) / n
se = sqrt.(abs.(diag(V)))


5-element Vector{Float64}:
      2.419705471706707e9
 908134.4935175716
      1.1687123578664e6
  71584.66265448726
 114191.97442222189

In [60]:
θ1

5-element Vector{Float64}:
  12.171440882361711
   0.3746643457826189
  -0.261800029522286
   0.9368520859660205
 -16.257097998844962

In [ ]:
#markup for leontieff

A= @. main.cost_m / main.rev
B= @. main.cost_l / (main.rev * beta_l_total_2)
markup_leontief= @. 1/(A+B)

mean_markup_leontief= mean(skipmissing(markup_leontief))
median_markup_leontief= median(skipmissing(markup_leontief))



2.771184441258282